In [41]:
from scipy.stats import norm
from math import log, sqrt, exp

def BlackScholes(CallPutFlag,S,K,r,v,T,d):
    """ # The Black Scholes Formula
    # CallPutFlag - This is set to 'c' for call option, anything else for put
    # S - Stock price
    # K - Strike price
    # T - Time to maturity
    # r - Riskfree interest rate
    # d - Dividend yield
    # v - Volatility
    """
    d1 = (log(float(S)/K)+((r-d)+v*v/2.)*T)/(v*sqrt(T))
    d2 = d1-v*sqrt(T)
    if CallPutFlag=='c':
        return S*exp(-d*T)*norm.cdf(d1)-K*exp(-r*T)*norm.cdf(d2)
    else:
        return K*exp(-r*T)*norm.cdf(-d2)-S*exp(-d*T)*norm.cdf(-d1)

#The Greeks
def Black_Scholes_Greeks_Call(S, K, r, v, T, d):
    """Calculating the partial derivatives for a Black Scholes Option (Call)
    # S - Stock price
    # K - Strike price
    # T - Time to maturity
    # r - Riskfree interest rate
    # d - Dividend yield
    # v - Volatility
    Return:
    Delta: partial wrt S
    Gamma: second partial wrt S
    Theta: partial wrt T
    Vega: partial wrt v
    Rho: partial wrt r """
    T_sqrt = sqrt(T)
    d1 = (log(float(S)/K)+((r-d)+v*v/2.)*T)/(v*T_sqrt)
    d2 = d1-v*T_sqrt
    Delta = norm.cdf(d1)
    Gamma = norm.pdf(d1)/(S*v*T_sqrt)
    Theta =- (S*v*norm.pdf(d1))/(2*T_sqrt) - r*K*exp( -r*T)*norm.cdf(d2)
    Vega = S * T_sqrt*norm.pdf(d1)
    Rho = K*T*exp(-r*T)*norm.cdf(d2)
    return Delta, Gamma, Theta, Vega, Rho

def Black_Scholes_Greeks_Put(S, K, r, v, T, d):
    """Calculating the partial derivatives for a Black Scholes Option (Put)
    # S - Stock price
    # K - Strike price
    # T - Time to maturity
    # r - Riskfree interest rate
    # d - Dividend yield
    # v - Volatility
    Return:
    Delta: partial wrt S
    Gamma: second partial wrt S
    Theta: partial wrt T
    Vega: partial wrt v
    Rho: partial wrt r """
    T_sqrt = sqrt(T)
    d1 = (log(float(S)/K)+r*T)/(v*T_sqrt) + 0.5*v*T_sqrt
    d2 = d1-(v*T_sqrt)
    Delta = -norm.cdf(-d1)
    Gamma = norm.pdf(d1)/(S*v*T_sqrt)
    Theta = -(S*v*norm.pdf(d1)) / (2*T_sqrt)+ r*K * exp(-r*T) * norm.cdf(-d2)
    Vega = S * T_sqrt * norm.pdf(d1)
    Rho = -K*T*exp(-r*T) * norm.cdf(-d2)
    return Delta, Gamma, Theta, Vega, Rho


In [42]:

print(Black_Scholes_Greeks_Call(100, 100, 0.005, 0.06, 0.4, 0))
print(Black_Scholes_Greeks_Put(100, 100, 0.005, 0.06, 0.4, 0))

print(BlackScholes('c', 100, 100, 0.005, 0.06, 0.4, 0))
print(BlackScholes('p', 100, 100, 0.005, 0.06, 0.4, 0))

(0.5285710345530259, 0.10486079971293322, -2.1437085315040076, 25.166591931103973, 20.49713093369678)
(-0.47142896544697405, 0.10486079971293322, -1.644707532170341, 25.166591931103973, -19.42294901299654)
1.6142761210606338
1.4144759877939421


In [33]:
def BinomialTree3(type,S0, K, r, sigma, T, N=2000,american="false"):
    #we improve the previous tree by checking for early exercise for american options
   
    #calculate delta T    
    deltaT = float(T) / N
 
    # up and down factor will be constant for the tree so we calculate outside the loop
    u = np.exp(sigma * np.sqrt(deltaT))
    d = 1.0 / u
 
    #to work with vector we need to init the arrays using numpy
    fs =  np.asarray([0.0 for i in range(N + 1)])
        
    #we need the stock tree for calculations of expiration values
    fs2 = np.asarray([(S0 * u**j * d**(N - j)) for j in range(N + 1)])
    
    #we vectorize the strikes as well so the expiration check will be faster
    fs3 =np.asarray( [float(K) for i in range(N + 1)])
    
 
    #rates are fixed so the probability of up and down are fixed.
    #this is used to make sure the drift is the risk free rate
    a = np.exp(r * deltaT)
    p = (a - d)/ (u - d)
    oneMinusP = 1.0 - p
 
   
    # Compute the leaves, f_{N, j}
    if type =="C":
        fs[:] = np.maximum(fs2-fs3, 0.0)
    else:
        fs[:] = np.maximum(-fs2+fs3, 0.0)
    
   
    #calculate backward the option prices
    for i in range(N-1, -1, -1):
       fs[:-1]=np.exp(-r * deltaT) * (p * fs[1:] + oneMinusP * fs[:-1])
       fs2[:]=fs2[:]*u
      
       if american=='true':
           #Simply check if the option is worth more alive or dead
           if type =="C":
                fs[:]=np.maximum(fs[:],fs2[:]-fs3[:])
           else:
                fs[:]=np.maximum(fs[:],-fs2[:]+fs3[:])
                
    # print fs
    return fs[0]

In [26]:
BlackScholes('c', 100, 100, 0.005, 0.06, 0, 0.4)

1.1432335348292781

In [30]:
BinomialTree3("C", 26.14, 26.15, 0.0641, 0.31, 0.996307, american="false")

3.9835172452950496

In [10]:
# valuing an American option

from QuantLib import *

valuation_date = Date(22, 8, 2018)
Settings.instance().evaluationDate = valuation_date+2

calendar = Canada()
volatility = 42.66/100
day_count = Actual365Fixed()

underlying = 13.5
risk_free_rate = 2.13/100
dividend_rate = 1.2/100

exercise_date = Date(22, 8, 2021)
strike = 13
option_type = Option.Put

payoff = PlainVanillaPayoff(option_type, strike)
exercise = EuropeanExercise(exercise_date)
european_option = VanillaOption(payoff, exercise)

spot_handle = QuoteHandle(SimpleQuote(underlying))

flat_ts = YieldTermStructureHandle(FlatForward(valuation_date,risk_free_rate,day_count))
dividend_yield = YieldTermStructureHandle(FlatForward(valuation_date,dividend_rate,day_count))
flat_vol_ts = BlackVolTermStructureHandle(BlackConstantVol(valuation_date,calendar,volatility,day_count))
bsm_process = BlackScholesMertonProcess(spot_handle,dividend_yield,flat_ts,flat_vol_ts)

# European option
european_option.setPricingEngine(AnalyticEuropeanEngine(bsm_process))
bs_price = european_option.NPV()
print("European option price is ", bs_price)

# American option
binomial_engine = BinomialVanillaEngine(bsm_process, "crr", 50)
am_exercise = AmericanExercise(valuation_date, exercise_date)
american_option = VanillaOption(payoff, am_exercise)
american_option.setPricingEngine(binomial_engine)
crr_price = american_option.NPV()
print("American option price is ", crr_price)

ImportError: No module named 'QuantLib'